# von Karman plate equation

\begin{align}
\Delta^2 w &= [w,F]-p\Delta w + f\\
\Delta^2 F &=-\frac12[w,w]+g
\end{align}
with $[u,v]:=u_{xx}v_{yy}+u_{yy}v_{xx}-2u_{xy}v_{xy} = \mathrm{cof}(\nabla^2 u):\nabla^2v$.

Weak form
\begin{align}
\int_{\Omega}\nabla^2 w:\nabla^2\delta w\,dx&= \int_{\Omega}[w,F]\delta w+p\nabla w\cdot\nabla\delta w + f\delta w\,dx\\
\int_{\Omega}\nabla^2 F:\nabla^2 \delta F\,dx &=\int_{\Omega}-\frac12[w,w]\delta F+g\delta F\,dx
\end{align}

Mixed form, $\sigma:=\nabla^2 w$, $\tau:=\nabla^2F$
\begin{align}
\int_{\Omega}\sigma:\delta\sigma\,dx + \langle \mathrm{div}(\delta\sigma),\nabla w\rangle&= 0\\
\langle\mathrm{div}(\sigma):\nabla\delta w\rangle&= -\int_{\Omega}\mathrm{cof}(\sigma):\tau\,\delta w+p\nabla w\cdot\nabla\delta w + f\delta w\,dx\\
\langle\mathrm{div}(\tau),\nabla \delta F\rangle &=\int_{\Omega}\frac12\mathrm{cof}(\sigma):\sigma\,\delta F-g\delta F\,dx\\
\int_{\Omega}\tau:\delta\tau\,dx + \langle \mathrm{div}(\delta\tau),\nabla F\rangle &=0
\end{align}


In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

In [2]:
def SolveKarman(mesh, force, p, dirichlet, g=CF(0), order=3):
    Q = H1(mesh, order=order, dirichlet=dirichlet)
    V = HDivDiv(mesh, order=order-1)
    X = Q*Q*V*V
    (w,F,sigma,tau), (dw,dF,dsigma,dtau) = X.TnT()

    n = specialcf.normal(2)

    def tang(u): return u-(u*n)*n

    B = BilinearForm(X, symmetric=True)
    B += (InnerProduct (sigma, dsigma) + div(sigma)*grad(dw) \
          + div(dsigma)*grad(w) - 1e-10*w*dw )*dx \
          + (-(sigma*n) * tang(grad(dw)) - (dsigma*n)*tang(grad(w)))*dx(element_boundary=True)
    B += (InnerProduct (tau, dtau) + div(tau)*grad(dF) \
          + div(dtau)*grad(F) - 1e-10*F*dF )*dx \
          + (-(tau*n) * tang(grad(dF)) - (dtau*n)*tang(grad(F)))*dx(element_boundary=True)
    B += (p*Grad(w)*Grad(dw)+InnerProduct(Cof(sigma),tau)*dw-0.5*InnerProduct(Cof(sigma),sigma)*dF + 
          force*dw + g*dF)*dx

    gfsol = GridFunction(X)

    solvers.Newton(B,gfsol,inverse="sparsecholesky")
    
    return gfsol

## Example with known solution
Adapt second equation to $\Delta^2 F =-\frac12[w,w]+g$ for generating right-hand side.

$w=x^2(1 − x)^2y^2(1 − y)^2$, $F=\sin(\pi x)^2\sin(\pi y)^2$

In [3]:
rect = Rectangle(1,1).Face()
mesh = Mesh(OCCGeometry(rect,dim=2).GenerateMesh(maxh=0.05))

wex = 100*x**2*(1-x)**2*y**2*(1-y)**2
Fex =sin(pi*x)**2*sin(pi*y)**2

p=0
gradwex = CF( (wex.Diff(x),wex.Diff(y)) )
hessewex = CF( (wex.Diff(x).Diff(x),wex.Diff(x).Diff(y),
                wex.Diff(y).Diff(x),wex.Diff(y).Diff(y)), dims=(2,2) )

gradFex = CF( (Fex.Diff(x),Fex.Diff(y)) )
hesseFex = CF( (Fex.Diff(x).Diff(x),Fex.Diff(x).Diff(y),
                Fex.Diff(y).Diff(x),Fex.Diff(y).Diff(y)), dims=(2,2) )
bilaplacewex = wex.Diff(x).Diff(x).Diff(x).Diff(x)+2*wex.Diff(x).Diff(x).Diff(y).Diff(y)+ \
               wex.Diff(y).Diff(y).Diff(y).Diff(y)
bilaplaceFex = Fex.Diff(x).Diff(x).Diff(x).Diff(x)+2*Fex.Diff(x).Diff(x).Diff(y).Diff(y)+ \
                Fex.Diff(y).Diff(y).Diff(y).Diff(y)

g = bilaplaceFex + 0.5*InnerProduct(Cof(hessewex),hessewex)
f = bilaplacewex - InnerProduct(Cof(hessewex),hesseFex) + p*Trace(hessewex)


with TaskManager():
    gfsol = SolveKarman(mesh, p=p, force=f, g=g, dirichlet=".*", order=1)
    
gfw,gfF,gfsigma,gftau = gfsol.components
Draw(gfw, mesh, name="u", deformation=True)
Draw(gfF, mesh, name="v", deformation=True)
# Draw(wex, mesh, deformation=True)
# Draw(Fex, mesh, deformation=True)
# Draw(mesh)

errw = sqrt(Integrate((gfw-wex)**2+(Grad(gfw)-gradwex)**2,mesh))
errF = sqrt(Integrate((gfF-Fex)**2+(Grad(gfF)-gradFex)**2,mesh))

print("errw = ", errw, ", errF = ", errF)

Newton iteration  0
err =  14.959454708365923
Newton iteration  1
err =  0.7924593739928854
Newton iteration  2
err =  0.14784946576077718
Newton iteration  3
err =  0.024732782741293945
Newton iteration  4
err =  0.003929593285029371
Newton iteration  5
err =  0.0006140118016371888
Newton iteration  6
err =  9.5514590457965e-05
Newton iteration  7
err =  1.4842182480773427e-05
Newton iteration  8
err =  2.305797916104826e-06
Newton iteration  9
err =  3.58197258209426e-07
Newton iteration  10
err =  5.564403899485182e-08
Newton iteration  11
err =  8.643984396959871e-09
Newton iteration  12
err =  1.3427929087409883e-09
Newton iteration  13
err =  2.0859659311552407e-10
Newton iteration  14
err =  3.240529859951347e-11
Newton iteration  15
err =  5.033291895153501e-12


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

errw =  0.0722836763352815 , errF =  0.17778891866393715


In [4]:
rect = Rectangle(1,1).Face()
mesh = Mesh(OCCGeometry(rect,dim=2).GenerateMesh(maxh=0.05))
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene